# SPACEX DASH WEB APP

In [1]:
# Import required libraries
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [2]:

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()
launch_sites = spacex_df['Launch Site'].unique()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(
                                    id='site-dropdown',
                                    options=[{'label': 'All Sites', 'value': 'ALL'}] + [
                                        {'label': site, 'value': site} for site in launch_sites
                                    ],
                                    value='ALL',
                                    placeholder='Select a Launch Site here',
                                    searchable=True,
                                ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(
                                    id='payload-slider',
                                    min=0,
                                    max=10000,
                                    step=1000,
                                    value=[min_payload, max_payload],
                                    marks={
                                        0: '0',
                                        2500: '2500',
                                        5000: '5000',
                                        7500: '7500',
                                        10000: '10000',
                                    },
                                ),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output('success-pie-chart', 'figure'), Input('site-dropdown', 'value'))
def update_success_pie(selected_site: str):
    """Render success distribution based on selected launch site."""
    if selected_site == 'ALL':
        # Sum successful launches per site to highlight top performer.
        success_counts = (
            spacex_df.groupby('Launch Site')['class']
            .sum()
            .reset_index(name='successes')
        )
        fig = px.pie(
            success_counts,
            values='successes',
            names='Launch Site',
            title='Total Success Launches By Site',
        )
    else:
        site_df = spacex_df[spacex_df['Launch Site'] == selected_site]
        outcome_counts = (
            site_df['class']
            .value_counts()
            .rename_axis('class')
            .reset_index(name='Count')
        )
        outcome_counts['Outcome'] = outcome_counts['class'].map({1: 'Success', 0: 'Failure'})
        fig = px.pie(
            outcome_counts,
            values='Count',
            names='Outcome',
            title=f'Total Success Launches for site {selected_site}',
            color='Outcome',
            color_discrete_map={
                'Success': '#EF553B',  # Highlight successful launches in red
                'Failure': '#636EFA',  # Highlight failed launches in blue
            },
            category_orders={'Outcome': ['Success', 'Failure']},
        )

    return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    Output('success-payload-scatter-chart', 'figure'),
    [
        Input('site-dropdown', 'value'),
        Input('payload-slider', 'value'),
    ],
)
def update_payload_scatter(selected_site: str, payload_range):
    """Scatter plot showing payload vs. success filtered by site and payload range."""
    low, high = payload_range
    filtered_df = spacex_df[
        (spacex_df['Payload Mass (kg)'] >= low)
        & (spacex_df['Payload Mass (kg)'] <= high)
    ]

    if selected_site != 'ALL':
        filtered_df = filtered_df[filtered_df['Launch Site'] == selected_site]
        title = f'Correlation between Payload and Success for site {selected_site}'
    else:
        title = 'Correlation between Payload and Success for all Sites'

    fig = px.scatter(
        filtered_df,
        x='Payload Mass (kg)',
        y='class',
        color='Booster Version Category',
        title=title,
        hover_data=['Launch Site'],
    )

    return fig


# Run the app
if __name__ == '__main__':
    app.run()


# Finding Insights Visually

## Now with the dashboard completed, you should be able to use it to analyze SpaceX launch data, and answer the following questions:

1. Which site has the largest successful launches?
    - KSC LC-39A has the most successful launches (10 successes out of 13 attempts, 76.9% success rate).

2. Which site has the highest launch success rate?
    - Same site (KSC LC-39A) also achieves the highest success rate among sites; 
    
    - the lowest rate is at CCAFS LC-40 (7/26, 26.9%).

3. Which payload range(s) has the highest launch success rate?
    - Payloads in the 7.5k–10k kg band show the strongest performance (3 successes in 5 launches, 60%); 

4. Which payload range(s) has the lowest launch success rate?
    - the weakest band is 5k–7.5k kg (2/9, 22%).

5. Which F9 Booster version (v1.0, v1.1, FT, B4, B5, etc.) has the highest
    launch success rate?
    - Booster version B5 records the top success rate (1/1 = 100%), though based on a single launch; 
    
    - considering larger samples, FT boosters perform best (16/24, 66.7%) followed by B4 (6/11, 54.5%).